In [1]:
import json
import sqlite3
import pandas as pd
import numpy as np
import pyvista as pv

In [2]:
lines = 3000
db_path = './RL_random_cogging.db'

In [3]:
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(f"SELECT * FROM strike LIMIT {int(lines)};", conn)

In [4]:
df

,step_number,position,rotation,result,series_id,press_id,temperature,score,score_type
0,1,"[-3.2027971744537354, 0.0, 0.0]","[0.668729985922069, 0.0, 0.0, 0.7435053503026521]","{""Vertices"": [3.818729283928602, 1.50759959728...",94b9ec21-6892-44b2-b404-7a4629df0feb,2,None,0.0,
1,2,"[-3.4823009967803955, 0.0, 0.0]","[0.014491463524945375, 0.0, 0.0, 0.99989499322...","{""Vertices"": [3.816492441285132, 1.46616065543...",94b9ec21-6892-44b2-b404-7a4629df0feb,2,None,0.0,
2,3,"[-0.0017781257629394531, 0.0, 0.0]","[0.8530763101481396, 0.0, 0.0, 0.5217861717830...","{""Vertices"": [3.8191772915308464, 1.4474579204...",94b9ec21-6892-44b2-b404-7a4629df0feb,2,None,0.0,
3,4,"[3.5798449516296387, 0.0, 0.0]","[0.44194763779898066, 0.0, 0.0, 0.897040849373...","{""Vertices"": [3.8246398523335445, 1.4455582531...",94b9ec21-6892-44b2-b404-7a4629df0feb,2,None,0.0,
4,5,"[-1.0711710453033447, 0.0, 0.0]","[0.9440819082168107, 0.0, 0.0, 0.3297110107013...","{""Vertices"": [3.8310217223481375, 1.4466357752...",94b9ec21-6892-44b2-b404-7a4629df0feb,2,None,0.0,
...,...,...,...,...,...,...,...,...,...
2995,146,"[3.706169843673706, 0.0, 0.0]","[0.9914036199489485, 0.0, 0.0, 0.1308390704343...","{""Vertices"": [4.77551641486954, 1.756692381430...",07f2cbcb-bdc2-45ee-978f-e70fa9795fc9,2,None,0.0,
2996,147,"[3.543666362762451, 0.0, 0.0]","[0.9999999879468104, 0.0, 0.0, 0.0001552622909...","{""Vertices"": [4.77551641486954, 1.756692381430...",07f2cbcb-bdc2-45ee-978f-e70fa9795fc9,2,None,0.0,
2997,148,"[3.7755520343780518, 0.0, 0.0]","[1.0, 0.0, 0.0, 6.123233995736766e-17]","{""Vertices"": [4.771084155276381, 1.78307114806...",07f2cbcb-bdc2-45ee-978f-e70fa9795fc9,2,None,0.0,
2998,149,"[3.77238130569458, 0.0, 0.0]","[1.0, 0.0, 0.0, 6.123233995736766e-17]","{""Vertices"": [4.771402716258504, 1.79561296509...",07f2cbcb-bdc2-45ee-978f-e70fa9795fc9,2,None,0.0,


In [32]:
record = json.loads(df.iloc[-100]['result'])
print(record.keys())

dict_keys(['Vertices', 'Triangles', 'Steps'])


In [33]:
record['Steps']

[0.03, 0.007636965513229371]

In [34]:
# 1) Vertices: reshape to (N, 3)
V_flat = np.asarray(record["Vertices"], dtype=float)       # shape (6120,)
assert V_flat.size % 3 == 0, "Vertices length must be a multiple of 3"
V = V_flat.reshape(-1, 3)                                  # (N, 3)

# 2) Triangles: reshape to (M, 3)
T_flat = np.asarray(record["Triangles"], dtype=int)        # shape (6108,)
assert T_flat.size % 3 == 0, "Triangles length must be a multiple of 3"
T = T_flat.reshape(-1, 3)                                  # (M, 3)

# 3) Build VTK faces array: [3, i, j, k, 3, i, j, k, ...]
faces = np.hstack([np.full((T.shape[0], 1), 3, dtype=np.int64), T.astype(np.int64)]).ravel()

# 4) Make mesh and plot
mesh = pv.PolyData(V, faces)
mesh.compute_normals(point_normals=True, cell_normals=True, inplace=True)
mesh.plot(smooth_shading=False, show_edges=True)

Widget(value='<iframe src="http://localhost:51431/index.html?ui=P_0x2078e9ccca0_7&reconnect=auto" class="pyvis…

In [ ]:
df.iloc[-100]['rotation']


'[0.8370419949238239, 0.0, 0.0, 0.547138646719408]'

In [50]:
df.iloc[-100]['position']

'[3.71505069732666, 0.0, 0.0]'